# **Part 1: Run MobileNet on GPU**

In this tutorial, we will explore how to train a neural network with PyTorch.

### Setup (5%)

We will first install a few packages that will be used in this tutorial and also define the path of CUDA library:

In [2]:
!pip install torchprofile 1>/dev/null
!ldconfig /usr/lib64-nvidia 2>/dev/null
!pip install onnx 1>/dev/null
!pip install onnxruntime 1>/dev/null

We will then import a few libraries:

In [3]:
import random

import numpy as np
import torch
import torchvision
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm

In [4]:
print(torch.__version__)
print(torchvision.__version__)

2.1.0+cu121
0.16.0+cu121


To ensure the reproducibility, we will control the seed of random generators:

In [5]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

We must decide the HYPER-parameter before training the model:

In [34]:
NUM_CLASSES = 10

# TODO:
# Decide your own hyper-parameters
BATCH_SIZE = 32
LEARNING_RATE = 0.01
NUM_EPOCH = 5

### Data  (5%)

In this lab, we will use CIFAR-10 as our target dataset. This dataset contains images from 10 classes, where each image is of
size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.

:Before using the data as input, we can do data pre-processing with transform function:

In [35]:
# TODO:
# Resize images to 224x224, i.e., the input image size of MobileNet,
# Convert images to PyTorch tensors, and
# Normalize the images with mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transform,
  )

Files already downloaded and verified
Files already downloaded and verified


To train a neural network, we will need to feed data in batches.

We create data loaders with the batch size determined previously in setup section:

In [36]:
# each dataflow is a dataloader
dataflow = {}
for split in ['train', 'test']:
  dataflow[split] = DataLoader(
    dataset[split],
    batch_size=BATCH_SIZE,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
    drop_last=True
  )

We can print the data type and shape from the training data loader:

In [37]:
for inputs, targets in dataflow["train"]:
  print(f"[inputs] dtype: {inputs.dtype}, shape: {inputs.shape}")
  print(f"[targets] dtype: {targets.dtype}, shape: {targets.shape}")
  #print(f'input: {inputs}')
  #print(f'targets: {targets}')
  break

[inputs] dtype: torch.float32, shape: torch.Size([32, 3, 224, 224])
[targets] dtype: torch.int64, shape: torch.Size([32])


### Model (10%)

In this tutorial, we will import MobileNet provided by torchvision, and use the pre-trained weight:

In [38]:
# TODO:
# Load pre-trained MobileNetV2
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
model = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
print(model)



MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

You should observe that the output dimension of the classifier does not match the number of cleasses in CIFAR-10.

Now change the output dimension of the classifer to number of classes:

In [39]:
# TODO:
# Change the output dimension of the classifer to number of classes

model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
print(model)

# Send the model from cpu to gpu
model = model.cuda()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

Now the output dimension of the classifer matches.

As this course focuses on efficiency, we will then inspect its model size and (theoretical) computation cost.


* The model size can be estimated by the number of trainable parameters:

In [40]:
num_params = 0
for param in model.parameters():
  if param.requires_grad:
    num_params += param.numel()
print("#Params:", num_params)

#Params: 2236682


* The computation cost can be estimated by the number of [multiply–accumulate operations (MACs)](https://en.wikipedia.org/wiki/Multiply–accumulate_operation) using [TorchProfile](https://github.com/zhijian-liu/torchprofile), we will further use this profiling tool in the future labs .

In [41]:
num_macs = profile_macs(model, torch.zeros(1, 3, 224, 224).cuda())
print("#MACs:", num_macs)

#MACs: 306186464


This model has 2.2M parameters and requires 306M MACs for inference. We will work together in the next few labs to improve its efficiency.

### Optimization (10%)

As we are working on a classification problem, we will apply [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) as our loss function to optimize the model:

In [42]:
# TODO:
# Apply cross entropy as our loss function
criterion = nn.CrossEntropyLoss()

We should decide an optimizer for the model:

In [43]:
# TODO:
# Choose an optimizer.
optimizer = SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

(Optional) We can apply a learning rate scheduler during the training:

In [44]:
# TODO(optional):
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

### Training (25%)

We first define the function that optimizes the model for one batch:

In [45]:
def train_one_batch(
  model: nn.Module,
  criterion: nn.Module,
  optimizer: Optimizer,
  #scheduler: LRScheduler,
  inputs: torch.Tensor,
  targets: torch.Tensor
) -> None:

  # TODO:
  # Step 1: Reset the gradients (from the last iteration)
  optimizer.zero_grad()
  # Step 2: Forward inference
  outputs = model(inputs)
  # Step 3: Calculate the loss
  loss = criterion(outputs, targets)
  # Step 4: Backward propagation
  loss.backward()
  # Step 5: Update optimizer
  optimizer.step()
  # (Optional Step 6: scheduler)
  #if scheduler is not None:
  #  scheduler.step()

We then define the training function:

In [46]:
def train(
    model: nn.Module,
    dataflow: DataLoader,
    criterion: nn.Module,
    optimizer: Optimizer,
    #scheduler: LRScheduler
):

  model.train()

  for inputs, targets in tqdm(dataflow, desc='train', leave=False):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Call train_one_batch function
    train_one_batch(model, criterion, optimizer, inputs, targets)


Last, we define the evaluation function:

In [47]:
def evaluate(
  model: nn.Module,
  dataflow: DataLoader
) -> float:

  model.eval()
  num_samples = 0
  num_correct = 0

  with torch.no_grad():
    for inputs, targets in tqdm(dataflow, desc="eval", leave=False):
      # TODO:
      # Step 1: Move the data from CPU to GPU
      inputs = inputs.cuda()
      targets = targets.cuda()
      # Step 2: Forward inference
      outputs = model(inputs)
      # Step 3: Convert logits to class indices (predicted class)
      _, predicts = torch.max(outputs, 1) # torch.max() return index 0: indice of maximum value, index 1: maximum value

      # Update metrics
      num_samples += targets.size(0)
      num_correct += (predicts == targets).sum()

  return (num_correct / num_samples * 100).item()

With training and evaluation functions, we can finally start training the model!

If the training is done properly, the accuracy should simply reach higher than 0.925:

In [48]:
for epoch_num in tqdm(range(1, NUM_EPOCH + 1)):
  train(model, dataflow["train"], criterion, optimizer)
  acc = evaluate(model, dataflow["test"])
  print(f"epoch {epoch_num}:", acc)

print(f"final accuracy: {acc}")

  0%|          | 0/5 [00:00<?, ?it/s]

train:   0%|          | 0/1562 [00:00<?, ?it/s]

eval:   0%|          | 0/312 [00:00<?, ?it/s]

epoch 1: 89.90384674072266


train:   0%|          | 0/1562 [00:00<?, ?it/s]

...

eval:   0%|          | 0/312 [00:00<?, ?it/s]

epoch 4: 93.359375


train:   0%|          | 0/1562 [00:00<?, ?it/s]

eval:   0%|          | 0/312 [00:00<?, ?it/s]

epoch 5: 92.99879455566406
final accuracy: 92.99879455566406


Save the weight of the model as "model.pt":

In [49]:
# TODO:
# Save the model weight
torch.save(model.state_dict(), "110611008_model.pt")

You will find "model.pt" in the current folder.

### Export Model (5%)

We can also save the model weight in [ONNX Format](https://pytorch.org/docs/stable/onnx_torchscript.html):

In [50]:
import torch.onnx

# TODO:
# Specify the input shape
dummy_input = torch.randn(BATCH_SIZE, 3, 224, 224)
dummy_input = dummy_input.cuda()
onnx_path = './model.onnx'

# TODO:
# Export the model to ONNX format
torch.onnx.export(model, dummy_input, onnx_path)

print(f"Model exported to {onnx_path}")

Model exported to ./model.onnx


In onnx format, we can observe the model structure using [Netron](https://netron.app/).

**Please download the model structure and hand in as YourID_onnx.png.**

### Inference (10%)

Load the saved model weight:



In [51]:
# TODO:
# Step 1: Get the model structure (mobilenet_v2 and the classifier)
loaded_model = torchvision.models.mobilenet_v2()
loaded_model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)

# Step 2: Load the model weight from "model.pt".
loaded_model.load_state_dict(torch.load("./110611008_model.pt"))

# Step 3: Send the model from cpu to gpu
loaded_model = loaded_model.cuda()


Run inference with the loaded model weight and check the accuracy:

In [52]:
acc = evaluate(loaded_model, dataflow["test"])
print(f"accuracy: {acc}")

eval:   0%|          | 0/312 [00:00<?, ?it/s]

accuracy: 92.99879455566406


If the accurracy is the same as the accuracy before saved, you have completed PART 1.

Congratulations!

# **Part 2: Training and Inference with torch.compile**

```torch.compile``` is a new feature in PyTorch 2.0.

The following tutorial will help you get to know the usage.

[Introduction to torch.compile](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)

### Training with torch.compile (15%)

We will reuse the ```train_one_batch``` function in the previous part.

To use ```torch.compile``` in training, we need to apply it on ```train_one_batch``` function:

In [53]:
import torch._dynamo
# Remind that whenever you use torch.compile, you need to use torch._dynamo.reset() to clear all compilation caches and restores the system to its initial state.
torch._dynamo.reset()

# TODO:
# Apply torch.compile on train_one_batch
train_one_batch_opt = torch.compile(train_one_batch, mode="reduce-overhead")

We define a new training function with ```train_one_batch_opt```:

In [54]:
def train_opt(
    model: nn.Module,
    dataflow: DataLoader,
    criterion: nn.Module,
    optimizer: Optimizer
):

  model.train()

  for inputs, targets in tqdm(dataflow, desc='train', leave=False):
    # TODO:
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Call train_one_batch_opt
    train_one_batch_opt(model, criterion, optimizer, inputs, targets)


We can observe that the training time of the first epoch is higher than the rest. The accuracy should easily be higher than 0.925, too:

In [56]:
# TODO:
# Prepare model
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
model = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)

# TODO:
# Send the model from cpu to gpu
model = model.cuda()

# TODO:
# Apply cross entropy as our loss function
criterion = nn.CrossEntropyLoss()

# TODO:
# Choose an optimizer.
optimizer = SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

for epoch_num in tqdm(range(1, NUM_EPOCH + 1)):
  train_opt(model, dataflow["train"], criterion, optimizer)
  acc = evaluate(model, dataflow["test"])
  print(f"epoch {epoch_num}:", acc)

print(f"final accuracy: {acc}")

  0%|          | 0/5 [00:00<?, ?it/s]

train:   0%|          | 0/1562 [00:00<?, ?it/s]

eval:   0%|          | 0/312 [00:00<?, ?it/s]

epoch 1: 90.48477172851562


train:   0%|          | 0/1562 [00:00<?, ?it/s]

...

eval:   0%|          | 0/312 [00:00<?, ?it/s]

epoch 4: 92.69831848144531


train:   0%|          | 0/1562 [00:00<?, ?it/s]

eval:   0%|          | 0/312 [00:00<?, ?it/s]

epoch 5: 92.97876739501953
final accuracy: 92.97876739501953


### Inference with torch.compile (15%)

Load the saved model trained from part 1 and apply torch.compile on the loaded model:

In [63]:
# TODO:
# Step 1: Get the model structure (mobilenet_v2 and the classifier)
loaded_model = torchvision.models.mobilenet_v2()
loaded_model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)

# Step 2: Load the model weight from "model.pt".
loaded_model.load_state_dict(torch.load("./110611008_model.pt"))

# Step 3: Send the model from cpu to gpu
loaded_model = loaded_model.cuda()
torch._dynamo.config.suppress_errors = True
torch._dynamo.reset()
# Step 4: Apply torch.compile on loaded_model
loaded_model_opt = torch.compile(loaded_model, mode="reduce-overhead")

# Inference
acc = evaluate(loaded_model_opt, dataflow["test"])
print(f"accuracy: {acc}")

eval:   0%|          | 0/312 [00:00<?, ?it/s]

[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING] WON'T CONVERT forward /usr/local/lib/python3.10/dist-packages/torchvision/models/mobilenetv2.py line 173 
[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING] due to: 
[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING] Traceback (most recent call last):
[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING]   File "/usr/lib/python3.10/concurrent/futures/process.py", line 720, in submit
[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING]     raise BrokenProcessPool(self._broken)
[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING] torch._dynamo.exc.BackendCompilerFailed: backend='inductor' raised:
[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING] BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore
[2024-02-22 16:33:37,811] torch._dynamo.convert_frame: [WARNING] 
[2024-02-22 16:33:37,811] torch._dynamo.conv

accuracy: 92.99879455566406


If the accurracy is the same as the accuracy in part 1, you have completed PART 2.

Congratulations!